--------------------------------------

# What’s a DSL?

  > A DSL is a programming language that's targeted at a specific problem; other programming languages that you use are more general purpose. It contains the syntax and semantics that model concepts at the same level of abstraction that the problem domain offers. 
  > Debashish Ghosh; DSLs in Action
  
  > The basic idea of a domain specific language (DSL) is a computer language that's targeted to a particular kind of problem, rather than a general purpose language that's aimed at any kind of software problem. Domain specific languages have been talked about, and used for almost as long as computing has been done.
  > Martin Fowler; https://martinfowler.com/bliki/DomainSpecificLanguage.html
  
  
![](images/DSLs.png)  
  
  
  
## Popular DSLs
  
  > DSLs are everywhere. Whether or not you brand them as DSLs, ... you’re using a lot of them in every application that you develop. 
  
DSL | Used for
:----|:---------
SQL | Relational database language used to query and manipulate data
Ant, Rake, Make | Languages for building software systems
CSS | Stylesheet description language
YACC, Bison, ANTLR | Parser-generator languages
RSpec, Cucumber | Behavior-driven testing language in Ruby
HTML | Markup language for the web

#### Which DSLs did you meet this semester?

#### Which of them were internal and which of them were external DSLs? 


## Execution model of a DSL

![](images/DSL_Execution_Model.png)


## Types of DSLs

### Internal DSLs

![](images/Internal_DSLs.png)

### External DSLs
![](images/External_DSLs.png)

# Miss Grant's Secret Compartment State Machine

![](https://www.digsdigs.com/photos/cool-gothic-living-room-designs-13.jpg)


![](images/Miss_Grant_SM.png)

## Being Software Engineer...
![](images/SM_class_diag.png)


## Creating a DSL

We will use a framework called Xtext to create external DSLs (http://www.eclipse.org/Xtext/).

### Writing a Language Grammar


```
grammar org.eclipse.xtext.example.fowlerdsl.Statemachine with org.eclipse.xtext.common.Terminals

generate statemachine "http://www.eclipse.org/xtext/example/fowlerdsl/Statemachine"

Statemachine :
     {Statemachine}
	('events' 
		events+=Event+ 
	'end')?
	('resetEvents' 
		resetEvents+=[Event]+ 
	'end')?
	('commands' 
		commands+=Command+ 
	'end')?
	states+=State*
;

Event:
	name=ID code=ID
;

Command:
	name=ID code=ID
;

State:
	'state' name=ID
		('actions' '{' actions+=[Command]+ '}')?
		transitions+=Transition*
	'end'
;

Transition:
	event=[Event] '=>' state=[State]
;
```


### Writing a Code Generator


```groovy
/*
 * generated by Xtext
 */
package org.eclipse.xtext.example.fowlerdsl.generator

import org.eclipse.emf.ecore.resource.Resource
import org.eclipse.xtext.example.fowlerdsl.statemachine.Command
import org.eclipse.xtext.example.fowlerdsl.statemachine.State
import org.eclipse.xtext.example.fowlerdsl.statemachine.Statemachine
import org.eclipse.xtext.generator.IFileSystemAccess
import org.eclipse.xtext.generator.IGenerator

/**
 * Generates code from your model files on save.
 * 
 * See https://www.eclipse.org/Xtext/documentation/303_runtime_concepts.html#code-generation
 */
class StatemachineGenerator implements IGenerator {
	
	override void doGenerate(Resource resource, IFileSystemAccess fsa) {
		fsa.generateFile(resource.className+".java", toJavaCode(resource.contents.head as Statemachine))
	}
	
	def className(Resource res) {
		var name = res.URI.lastSegment
		return name.substring(0, name.indexOf('.'))
	}
	
	def toJavaCode(Statemachine sm) '''
		import java.io.BufferedReader;
		import java.io.IOException;
		import java.io.InputStreamReader;
		
		public class «sm.eResource.className» {
			
			public static void main(String[] args) {
				new «sm.eResource.className»().run();
			}
			
			«FOR c : sm.commands»
				«c.declareCommand»
			«ENDFOR»
			
			protected void run() {
				boolean executeActions = true;
				String currentState = "«sm.states.head?.name»";
				String lastEvent = null;
				while (true) {
					«FOR state : sm.states»
						«state.generateCode»
					«ENDFOR»
					«FOR resetEvent : sm.resetEvents»
						if ("«resetEvent.name»".equals(lastEvent)) {
							System.out.println("Resetting state machine.");
							currentState = "«sm.states.head?.name»";
							executeActions = true;
						}
					«ENDFOR»
					
				}
			}
			
			private String receiveEvent() {
				System.out.flush();
				BufferedReader br = new BufferedReader(new InputStreamReader(System.in));
				try {
					return br.readLine();
				} catch (IOException ioe) {
					System.out.println("Problem reading input");
					return "";
				}
			}
		}
	'''
	
	def declareCommand(Command command) '''
		protected void do«command.name.toFirstUpper»() {
			System.out.println("Executing command «command.name» («command.code»)");
		}
	'''
	
	def generateCode(State state) '''
		if (currentState.equals("«state.name»")) {
			if (executeActions) {
				«FOR c : state.actions»
					do«c.name.toFirstUpper»();
				«ENDFOR»
				executeActions = false;
			}
			System.out.println("Your are now in state '«state.name»'. Possible events are [«
				state.transitions.map(t | t.event.name).join(', ')»].");
			lastEvent = receiveEvent();
			«FOR t : state.transitions»
				if ("«t.event.name»".equals(lastEvent)) {
					currentState = "«t.state.name»";
					executeActions = true;
				}
			«ENDFOR»
		}
	'''
}
```


# Your turn!

![](images/your_turn.gif)

## Setup Your Environment and Test it

### Installation of Eclipse Modelling Tools

For this lecture, we need Eclispe to work with the DSL Tools. Install it as in the following:

  * Navigate to http://www.eclipse.org.
  * Hit the `Download` button and download the Eclipse installer for your architecture.
      ![](./images/eclipse_download.png)
  * After downloading, execute the installer and choose to install the `Eclipse DSL Tools`.
      ![](./images/eclipse_install.png)
  * Subsequently, start Eclipse and navigate to the workbench-
      ![](./images/eclipse_wb.png)
  * Check that everything is working, by creating a first new language project.
      ![](./images/eclipse_wb.png)
      
Now, try if you can create an example "Hello World" language, as described below.

## Create an External DSL with Xtext

  * Create your own DSL for specification of Docker containers or VMs.
  * Create your own DSL for specification of Dockerized applications.
  * Create your own DSL for specification of of Docker Swarm deployments.
  * Create an alternative DSL for application deployment.
  
## Create a code generator with Xtend

  * Make the DSL that you created above executable by wrinting a code generator with Xtend.

# Create your own DSL

  * Create a new DSL with Xtext, by right-click in the `Package Explorer`
    ![](images/eclipse_new.png)
  * Choose `New->Other->Xtext Project` and click `Next`
    ![](images/eclipse_new_project.png)
    ![](images/eclipse_new_project2.png)
  * For the moment just keep the default values, i.e., name your language `mydsl`, and click `Finsish` subsequently.
    ![](images/eclipse_new_project3.png)
  * Now, create an editor, code validators, code completion, etc. Right-click on `GenerateMyDsl.mwe2 -> Run As -> MWE2 Workflow`
    ![](images/eclipse_run_mwe.png)
  * After all code was generated, start a new instance of Eclipse to try your small _Hello World_ Language
    ![](images/eclipse_run_instance.png)
  * In the new Eclipse instance, create a new project called `MyLangTest`
    ![](images/eclipse_instance_new_project.png)
    ![](images/eclipse_instance_new_project2.png)
    ![](images/eclipse_instance_new_project3.png)
  * In the newly created project, create a new file with the file ending that you specified earlier, here `*.mydsl`
    ![](images/eclipse_instance_new_file.png)
    ![](images/eclipse_instance_new_file2.png)
    ![](images/eclipse_instance_new_file_edit.png)
